# tests.ipynb

Notebook-based validation (Task: Unit Tests)

Includes:
- Correctness checks for Naive vs Windowed on a tiny synthetic dataset
- Performance check for windowed (optimized) strategy on 100k ticks (< 1s, < 100MB) using `time.perf_counter` and `tracemalloc`.



In [ ]:
%pip -q install pandas

In [ ]:
%run ./data_loader.ipynb
%run ./strategies.ipynb

In [ ]:
from datetime import datetime, timedelta
import time
import tracemalloc

## Correctness: moving averages match expected

In [ ]:
def make_ticks(prices):
    base = datetime(2026, 1, 1, 9, 30, 0)
    ticks = []
    for i, p in enumerate(prices):
        ticks.append(MarketDataPoint(timestamp=base + timedelta(seconds=i), symbol="GLD", price=float(p)))
    return ticks

ticks = make_ticks([10, 20, 30])

naive = NaiveMovingAverageStrategy()
w2 = WindowedMovingAverageStrategy(window_size=2)

ma1 = naive.generate_signals(ticks[0])[0]["moving_average"]
ma2 = naive.generate_signals(ticks[1])[0]["moving_average"]
ma3 = naive.generate_signals(ticks[2])[0]["moving_average"]

assert abs(ma1 - 10.0) < 1e-9
assert abs(ma2 - 15.0) < 1e-9
assert abs(ma3 - 20.0) < 1e-9

ma1w = w2.generate_signals(ticks[0])[0]["moving_average"]  # [10]
ma2w = w2.generate_signals(ticks[1])[0]["moving_average"]  # [10,20]=15
ma3w = w2.generate_signals(ticks[2])[0]["moving_average"]  # [20,30]=25

assert abs(ma1w - 10.0) < 1e-9
assert abs(ma2w - 15.0) < 1e-9
assert abs(ma3w - 25.0) < 1e-9

print("Correctness tests passed ✅")

## Performance: optimized/windowed under 1 second and <100MB for 100k ticks

In [ ]:
# Load real data
data = load_gld_market_data("GLD_market_data.csv")
assert len(data) >= 100_000, "CSV must have at least 100k rows for this test."

ticks = data[:100_000]
s = WindowedMovingAverageStrategy(window_size=10)

tracemalloc.start()
t0 = time.perf_counter()
for t in ticks:
    s.generate_signals(t)
elapsed = time.perf_counter() - t0
_, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

peak_mb = peak / (1024 * 1024)

print(f"Elapsed: {elapsed:.4f}s")
print(f"Peak traced memory: {peak_mb:.2f} MB")

assert elapsed < 1.0, f"Runtime too slow: {elapsed:.4f}s (expected < 1.0s)"
assert peak_mb < 100.0, f"Memory too high: {peak_mb:.2f}MB (expected < 100MB)"

print("Performance test passed ✅")

## Profiling output sanity check (hotspot expectation)

We expect Naive to spend time in `sum(...)` / moving average recomputation; windowed should not.

In [ ]:
import cProfile, pstats, io

def run_strategy(strategy, ticks):
    for t in ticks:
        strategy.generate_signals(t)

# small run for cProfile sanity
sub = data[:10_000]

pr = cProfile.Profile()
pr.enable()
run_strategy(NaiveMovingAverageStrategy(), sub)
pr.disable()

buf = io.StringIO()
pstats.Stats(pr, stream=buf).strip_dirs().sort_stats("cumtime").print_stats(20)
txt = buf.getvalue()
print(txt[:1500])

# very lightweight sanity check: 'sum' should often appear for naive
assert "sum" in txt, "Expected 'sum' to appear as a hotspot for naive strategy."

print("Hotspot sanity check passed ✅")